# 💒 Mobile Price 예측 모델링 - simple version
- Mobile_Price(Classification).ipynb 파일과 같은 과제를 수행합니다.
- 변수를 유형에 따라 분리하는 단계와 스케일링 수행 단계를 생략하여 전체 코드를 간략하게 줄였습니다.

### 💒 문제
- 주어진 학습용 데이터 X_train.csv를 활용하여 모바일 폰의 가격 범위(price_range)를 예측하는 모형을 만들고, 평가용 데이터 X_test.csv에 적용하여 가격 범위 예측값을 확률로 만들어 csv 파일로 생성하시오.(평가 지표는 ROC-AUC)

### 💒 힌트
- 분류 과제인지 회귀 과제인지 우선 파악해야 합니다.
- 평가 지표로 ROC-AUC를 사용하라고 한 것으로 보아 분류과제입니다. 이에 맞는 예측 모형으로 분류에 적합한 모형을 선정해야 합니다.
- 분류 과제에서는 pred와 proba가 모두 사용될 수 있으므로 무엇을 도출해야 하는지 파악합니다. 여기서는 확률을 요구하고 있으므로 proba를 사용합니다. 
- proba로 확률값을 요구하는 경우에는 평가지표로 ROC-AUC가 사용됩니다.

# 1. 필요한 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# 2. 데이터 읽어오기

In [2]:
X_train = pd.read_csv("./Mobile_price_X_train.csv")
X_test = pd.read_csv("./Mobile_price_X_test.csv")
y_train = pd.read_csv("./Mobile_price_y_train.csv")

# 3. info() 함수로 기본 정보 확인

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  998 non-null    float64
 1   clock_speed    1000 non-null   float64
 2   fc             1000 non-null   int64  
 3   int_memory     1000 non-null   int64  
 4   m_dep          1000 non-null   float64
 5   mobile_wt      1000 non-null   int64  
 6   n_cores        1000 non-null   int64  
 7   pc             1000 non-null   int64  
 8   px_height      1000 non-null   int64  
 9   px_width       1000 non-null   int64  
 10  ram            1000 non-null   int64  
 11  sc_h           1000 non-null   int64  
 12  sc_w           1000 non-null   int64  
dtypes: float64(3), int64(10)
memory usage: 101.7 KB


In [4]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  500 non-null    int64  
 1   clock_speed    500 non-null    float64
 2   fc             500 non-null    int64  
 3   int_memory     500 non-null    int64  
 4   m_dep          500 non-null    float64
 5   mobile_wt      500 non-null    int64  
 6   n_cores        500 non-null    int64  
 7   pc             500 non-null    int64  
 8   px_height      500 non-null    int64  
 9   px_width       500 non-null    int64  
 10  ram            500 non-null    int64  
 11  sc_h           500 non-null    int64  
 12  sc_w           500 non-null    int64  
dtypes: float64(2), int64(11)
memory usage: 50.9 KB


In [5]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   price_range  1000 non-null   int64
dtypes: int64(1)
memory usage: 7.9 KB


# 4. 결측치 처리하기
- 여기서는 간단한 수행을 위해 dropna()로 결측치가 포함된 모든 행을 제거하였습니다.

In [6]:
train = pd.concat([X_train, y_train], axis=1)
train = train.dropna()
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 998 entries, 0 to 999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  998 non-null    float64
 1   clock_speed    998 non-null    float64
 2   fc             998 non-null    int64  
 3   int_memory     998 non-null    int64  
 4   m_dep          998 non-null    float64
 5   mobile_wt      998 non-null    int64  
 6   n_cores        998 non-null    int64  
 7   pc             998 non-null    int64  
 8   px_height      998 non-null    int64  
 9   px_width       998 non-null    int64  
 10  ram            998 non-null    int64  
 11  sc_h           998 non-null    int64  
 12  sc_w           998 non-null    int64  
 13  price_range    998 non-null    int64  
dtypes: float64(3), int64(11)
memory usage: 117.0 KB


# 5. describe() 함수로 요약 통계량 확인하기

In [7]:
train.describe()

,battery_power,clock_speed,fc,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,price_range
count,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000,998.000000
mean,1173.311623,1.518337,4.203407,31.628257,0.506914,140.542084,4.447896,9.737475,600.136273,1200.589178,1233.038076,12.273547,5.617234,0.500000
std,428.594180,0.831860,4.331280,18.040050,0.288012,36.030157,2.292281,6.110091,412.413027,426.424588,611.904030,4.217545,4.208196,0.500251
min,501.000000,0.500000,0.000000,2.000000,0.100000,80.000000,1.000000,0.000000,0.000000,500.000000,256.000000,5.000000,0.000000,0.000000
25%,808.250000,0.600000,1.000000,16.000000,0.200000,109.000000,2.000000,4.000000,276.500000,831.000000,704.250000,9.000000,2.000000,0.000000
50%,1138.000000,1.500000,3.000000,31.000000,0.500000,141.000000,4.000000,10.000000,518.000000,1183.500000,1209.500000,12.000000,5.000000,0.500000
75%,1522.750000,2.300000,7.000000,47.000000,0.800000,172.000000,6.000000,15.000000,879.750000,1533.500000,1700.500000,16.000000,8.000000,1.000000
max,1996.000000,3.000000,19.000000,64.000000,1.000000,200.000000,8.000000,20.000000,1914.000000,1998.000000,2811.000000,19.000000,18.000000,1.000000


- concat으로 합친 train 데이터셋을 다시 원래의 컬럼대로 분리합니다.

In [8]:
x_train = train[['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep',
       'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h',
       'sc_w']]
y_train = train[['price_range']]

# 6. 학습용 데이터셋과 검증용 데이터셋 분리

In [9]:
from sklearn.model_selection import train_test_split

x_tr, x_valid, y_tr, y_valid = train_test_split(x_train, y_train['price_range'], 
                                                test_size = 0.3, stratify = y_train['price_range'])

# 7. 모델 학습

In [10]:
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(random_state = 42)
modelRF.fit(x_tr, y_tr)

RandomForestClassifier(random_state=42)

# 8. 모델 평가

In [11]:
y_validation_pred_proba = modelRF.predict_proba(x_valid)

In [12]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_valid, y_validation_pred_proba[:,1])
print(roc_auc)

0.9843999999999999


- validation으로 평가한 결과, 결측치를 대체하고 스케일링을 수행했던 기존 버전보다 더 높은 성능입니다. 😂
- 하이퍼 파라미터로 튜닝을 하거나 여러 단계로 스케일링을 거친 이후에 오히려 성능이 떨어질 수 있다는 것을 알 수 있습니다.😂

# 9. 파일 제출

In [13]:
pred = modelRF.predict_proba(X_test)[:,1]
result = pd.DataFrame({'pred':pred}).to_csv('수험번호.csv', index=False)
df = pd.read_csv('./수험번호.csv')
df

,pred
0,0.75
1,0.91
2,0.10
3,0.92
4,0.92
...,...
495,0.81
496,0.23
497,0.25
498,0.98
